In [1]:
import sys

sys.path.append("../")

import os
import numpy as np
import cv2
import pandas as pd

from pedrec.configs.dataset_configs import get_sim_dataset_cfg_default
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.datasets.pedrec_temporal_dataset import PedRecTemporalDataset
from pedrec.models.constants.action_mappings import ACTION
from pedrec.configs.dataset_configs import get_sim_dataset_cfg_default, PedRecTemporalDatasetConfig
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.datasets.pedrec_temporal_dataset import PedRecTemporalDataset
from pedrec.models.constants.action_mappings import ACTION
from pedrec.models.constants.sample_method import SAMPLE_METHOD
from pedrec.utils.augmentation_helper import get_affine_transform, get_affine_transforms
from pedrec.configs.app_config import AppConfig, action_list_c01
from pedrec.models.data_structures import ImageSize
from torch.utils.data import DataLoader, ConcatDataset
from random import randint
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
dataset_dir = "data/datasets/Conti01/"
dataset_file_train = "rt_conti_01_train_FIN.pkl"
dataset_file = "rt_conti_01_val_FIN.pkl"
dataset_result_file = "C01F_pred_df_experiment_pedrec_p2d3d_c_o_h36m_sim_mebow_0_allframes.pkl"
model_input_size = ImageSize(width=192, height=256)
cfg = PedRecNet50Config()
app_cfg = AppConfig()


In [3]:

dataset_cfg = PedRecTemporalDatasetConfig(
    flip=True,
    scale_factor=0.25,
    rotation_factor=0,
    skeleton_3d_range=3000,
    img_pattern="view_{cam_name}-frame_{id}.{type}",
    subsample=1,
    subsampling_strategy=SAMPLE_METHOD.SYSTEMATIC,
    gt_result_ratio=1,
    use_unit_skeleton=True,
    min_joint_score=0,
    add_2d=False
)

In [3]:
dataset = PedRecTemporalDataset(dataset_dir,
                                  dataset_file,
                                  DatasetType.VALIDATE,
                                  dataset_cfg,
                                  app_cfg.inference.action_list,
                                  None,
                                  pose_results_file=dataset_result_file)
dataset_length = len(dataset)



In [ ]:
ehpi_3d, action_id, annotations = dataset[50]
img_path = annotations.img_path
img_path = os.path.join(dataset_dir, img_path)
img_o = cv2.imread(img_path)
trans, trans_inv = get_affine_transforms(annotations.center, annotations.scale, 0, model_input_size, add_inv=True)
img_o = cv2.warpAffine(
    img_o,
    trans,
    (int(model_input_size.width), int(model_input_size.height)),
    flags=cv2.INTER_LINEAR)
fig, ax = plt.subplots()
ax.imshow(img_o)


In [ ]:
fig, ax = plt.subplots(6,3, figsize=(15,15)) 

count = 0
for i in range(0, 3):
    for j in range(0, 3):
        # TODO: Dazu noch das Bild
        entry = dataset[randint(0, dataset_length)]
        # entry = dataset[count + 606]
        ehpi_3d, action_id, annotations = entry
        img_path = annotations.img_path
        img_path = os.path.join(dataset_dir, img_path)
        img_o = cv2.imread(img_path)
        trans, trans_inv = get_affine_transforms(annotations.center, annotations.scale, 0, model_input_size, add_inv=True)
        img_o = cv2.warpAffine(
            img_o,
            trans,
            (int(model_input_size.width), int(model_input_size.height)),
            flags=cv2.INTER_LINEAR)

        actions = ""
        for idx, is_active in enumerate(action_id):
            if is_active == 1:
                actions = f"{actions}{app_cfg.inference.action_list[idx].name}, "
        img = ehpi_3d
        ax[i*2, j].imshow(img_o)
        ax[i*2+1, j].imshow(img)
        ax[i*2, j].set_title(actions)

In [4]:
fig, ax = plt.subplots(8,4, figsize=(15,15)) 
action_examples = []
count = 0
i = 0
j = 0
while True:
    if len(action_examples) == len(action_list_c01):
        break
    if count >= len(dataset):
        print("missing a action")
        break
    entry = dataset[randint(0, dataset_length)]
    # entry = dataset[count + 606]
    ehpi_3d, action_ids, annotations = entry
    if np.sum(action_ids) > 1:
        count += 1
        continue
    action = None
    for idx, is_active in enumerate(action_ids):
        if is_active == 1:
            action = app_cfg.inference.action_list[idx].name
            break
    assert action is not None
    if action in action_examples:
        count += 1
        continue
    action_examples.append(action)
    img_path = annotations.img_path
    img_path = os.path.join(dataset_dir, img_path)
    img_o = cv2.imread(img_path)
    trans, trans_inv = get_affine_transforms(annotations.center, annotations.scale, 0, model_input_size, add_inv=True)
    img_o = cv2.warpAffine(
        img_o,
        trans,
        (int(model_input_size.width), int(model_input_size.height)),
        flags=cv2.INTER_LINEAR)

    ax[i*2, j].imshow(img_o)
    ax[i*2+1, j].imshow(ehpi_3d)
    ax[i*2, j].set_title(action)
    count += 1
    i += 1
    if i > 3:
        j += 1
        i = 0
    ehpi = cv2.resize(ehpi_3d, (320, 320), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(f"data/models/ehpi3d/examples/{action}_bb.jpg", img_o)
    cv2.imwrite(f"data/models/ehpi3d/examples/{action}_ehpi3d.png", ehpi)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/dennis/code/python/pedrec/notebooks/../pedrec/utils/skeleton_helper_3d.py:41: RuntimeWarning: invalid value encountered in true_divide
  normalized_direction = direction / np.linalg.norm(direction, axis=1, keepdims=True)


IndexError: list index out of range

In [ ]:
dataset_df_path = os.path.join(dataset_dir, dataset_file)
df = pd.read_pickle(dataset_df_path)
pd.options.display.max_columns = None
pd.options.display.float_format= '{:.2f}'.format
print(f"Number of entrys: {df.shape[0]}")

In [ ]:
dataset_df_result_path = os.path.join(dataset_dir, dataset_result_file)
df_result = pd.read_pickle(dataset_df_result_path)
print(f"Number of entrys: {df_result.shape[0]}")

In [ ]:
filter_skeleton2d = [col for col in df if col.startswith('skeleton2d')]
filter_skeleton3d = [col for col in df if col.startswith('skeleton3d')]
filter_bb = [col for col in df if col.startswith('bb')]
filter_body_orientation = [col for col in df if col.startswith('body_orientation')]
filter_head_orientation = [col for col in df if col.startswith('head_orientation')]
filter_env = [col for col in df if col.startswith('env')]

In [ ]:
# skeleton 2d calculated columns
skeleton2d_xs = [col for col in df if col.startswith('skeleton2d') and col.endswith('_x')]
skeleton2d_ys = [col for col in df if col.startswith('skeleton2d') and col.endswith('_y')]
df["skeleton2d_width"] = df[skeleton2d_xs].max(axis=1) - df[skeleton2d_xs].min(axis=1)
df["skeleton2d_height"] = df[skeleton2d_ys].max(axis=1) - df[skeleton2d_ys].min(axis=1)

In [ ]:
# skeleton 3d calculated columns
skeleton3d_xs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_x')]
skeleton3d_ys = [col for col in df if col.startswith('skeleton3d') and col.endswith('_y')]
skeleton3d_zs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_z')]
df["skeleton3d_width"] = df[skeleton3d_xs].max(axis=1) - df[skeleton3d_xs].min(axis=1)
df["skeleton3d_height"] = df[skeleton3d_ys].max(axis=1) - df[skeleton3d_ys].min(axis=1)
df["skeleton3d_depth"] = df[skeleton3d_zs].max(axis=1) - df[skeleton3d_zs].min(axis=1)

In [ ]:
df.head(5)

In [ ]:
dataset_file = "rt_conti_01_train.pkl"  # report only frames with skeletons inside
df_val = pd.read_pickle(os.path.join(dataset_dir, dataset_file))

In [ ]:
action_counter = {}
for action_, movement_ in zip(df['actions'].iteritems(), df['movement'].iteritems()):
    actions = action_[1]
    movement = movement_[1]
    for action_id in actions:
        the_action = action_id
        if action_id == ACTION.WALK.value:
            if movement == ACTION.JOG.value:
                the_action = ACTION.JOG.value
        if the_action not in action_counter:
            action_counter[the_action] = [0, 0]
        action_counter[the_action][0] += 1

In [ ]:
df_val = pd.read_pickle(os.path.join(dataset_dir, dataset_file.replace("train", "val")))

In [ ]:
for action_, movement_ in zip(df_val['actions'].iteritems(), df_val['movement'].iteritems()):
    actions = action_[1]
    movement = movement_[1]
    for action_id in actions:
        the_action = action_id
        if action_id == ACTION.WALK.value:
            if movement == ACTION.JOG.value:
                the_action = ACTION.JOG.value
        if the_action not in action_counter:
            action_counter[the_action] = [0, 0]
        action_counter[the_action][1] += 1

In [ ]:
# print(sorted(action_counter.keys()))
for action_id, action_counts in sorted(action_counter.items()):
    print(f"{ACTION(action_id).name.replace('_', ' ')} & {action_counts[0]} & {action_counts[1]} \\\\")

In [6]:
def save_example(action):
  dataset = PedRecTemporalDataset(dataset_dir,
                                  dataset_file,
                                  DatasetType.VALIDATE,
                                  dataset_cfg,
                                  app_cfg.inference.action_list,
                                  None,
                                  action_filters=action.value,
                                  pose_results_file=None)
  dataset_length = len(dataset)
  entry = dataset[400]
  ehpi_3d, action_ids, annotations = entry
  action = ""
  for idx, is_active in enumerate(action_ids):
      if is_active == 1:
          action = f"{action}{app_cfg.inference.action_list[idx].name}_"
  assert action is not None
  action = action[:-1]
  img_path = annotations.img_path
  img_path = os.path.join(dataset_dir, img_path)
  img_o = cv2.imread(img_path)
  trans, trans_inv = get_affine_transforms(annotations.center, annotations.scale, 0, model_input_size, add_inv=True)
  img_o = cv2.warpAffine(
      img_o,
      trans,
      (int(model_input_size.width), int(model_input_size.height)),
      flags=cv2.INTER_LINEAR)

  ehpi = cv2.resize(ehpi_3d, (320, 320), interpolation=cv2.INTER_NEAREST)
  # print(ehpi_3d)
  print(action_ids)
  cv2.imwrite(f"data/models/ehpi3d/examples/{action}_bb.jpg", img_o)
  cv2.imwrite(f"data/models/ehpi3d/examples/{action}_ehpi3d.png", ehpi)
# save_example(ACTION.FIGHT)
# save_example(ACTION.THROW)
# save_example(ACTION.KICK_BALL)
# save_example(ACTION.ARGUE)
# save_example(ACTION.OPEN_DOOR)
# save_example(ACTION.WALK)
# save_example(ACTION.WAVE)
save_example(ACTION.STAND)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [16]:
val_dataset = PedRecTemporalDataset(dataset_dir,
                                  dataset_file,
                                  DatasetType.VALIDATE,
                                  dataset_cfg,
                                  app_cfg.inference.action_list,
                                  None)
train_dataset = PedRecTemporalDataset(dataset_dir, dataset_file_train, DatasetType.VALIDATE, dataset_cfg, app_cfg.inference.action_list, None)
# dataset = ConcatDataset([train_dataset, val_dataset])
# dataset_length = len(dataset)
# dataset = val_dataset

In [17]:
action_counter_val = np.array([0] * len(app_cfg.inference.action_list), dtype=np.float32)
for _, label in val_dataset:
  action_counter_val += label

/home/dennis/code/python/pedrec/notebooks/../pedrec/utils/skeleton_helper_3d.py:41: RuntimeWarning: invalid value encountered in true_divide
  normalized_direction = direction / np.linalg.norm(direction, axis=1, keepdims=True)


In [18]:
action_counter_train = np.array([0] * len(app_cfg.inference.action_list), dtype=np.float32)
for _, label in train_dataset:
  action_counter_train += label

In [20]:
for action_key, counts in enumerate(zip(action_counter_train, action_counter_val)):
  count_train, count_val = counts
  print(f"{app_cfg.inference.action_list[action_key].name} & ${int(count_train)}$ & ${int(count_val)}$ \\\\")

STAND & $409050$ & $45652$ \\
IDLE & $58206$ & $10168$ \\
WALK & $501943$ & $57699$ \\
JOG & $235386$ & $16207$ \\
WAVE & $47666$ & $2970$ \\
KICK_BALL & $9451$ & $1144$ \\
THROW & $8351$ & $1024$ \\
LOOK_FOR_TRAFFIC & $131570$ & $14819$ \\
HITCHHIKE & $38288$ & $4018$ \\
TURN_AROUND & $37380$ & $3494$ \\
WORK & $34580$ & $4263$ \\
ARGUE & $13055$ & $961$ \\
STUMBLE & $4967$ & $837$ \\
OPEN_DOOR & $13342$ & $1192$ \\
FALL & $8071$ & $591$ \\
STAND_UP & $6713$ & $873$ \\
FIGHT & $15512$ & $990$ \\
SIT & $0$ & $0$ \\
JUMP & $0$ & $0$ \\
WAVE_CAR_OUT & $0$ & $0$ \\
